In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

# GM

In [3]:
#you can also query like this
blaze_gm_data = pd.read_sql_query("""
with base as
(
   select
      *,
      last_day(week_start) year_month,
      retail_value / quantity_sold per_unit_price,
      cogs / quantity_sold per_unit_cost,
      sales.retail_value - sales.total_discount + sales.total_tax sales_post_tax_post_discount,
      sales.retail_value - sales.total_discount sales_pre_tax_post_discount,
      sales.retail_value sales_pre_tax_pre_discount,
   from
      blaze_data.sales sales
      -- EXCLUDE SAMPLES,DISPLAY,PROMO and test
   where
      lower(product_category) not like '%sample%'
      and lower(product_category) not like '%display%'
      and lower(product_category) not like '%promo%'
      and lower(product_category) not like '%test%'
    and state = 'MA'
)
,
monthly_location_rev_and_cogs as
(
   select
      year_month,
      shop_id shopid,
      company_id companyid,
      sum(sales_post_tax_post_discount) rev_post_tax_post_discount,
      sum(sales_pre_tax_post_discount) rev_pre_tax_post_discount,
      sum(sales_pre_tax_pre_discount) rev_pre_tax_pre_discount,
      sum(cogs) cogs
   from
      base
      -- EXCLUDE low COGS items
   where
      cogs > 1
      -- EXCLUDE high cost, high price items
      and per_unit_price <= 500
      and per_unit_cost <= 500
   group by
      2,
      3,
      1
   order by
      2,
      3,
      1
)
,
monthly_location_rev_and_cogs_and_profit as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      rev_post_tax_post_discount - cogs profit_post_tax_post_discount,
      rev_pre_tax_post_discount - cogs profit_pre_tax_post_discount,
      rev_pre_tax_pre_discount - cogs profit_pre_tax_pre_discount,
   from
      monthly_location_rev_and_cogs
)
,
monthly_location_rev_and_cogs_and_profit_and_margin as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      profit_post_tax_post_discount,
      profit_pre_tax_post_discount,
      profit_pre_tax_pre_discount,
      -- margin
      profit_post_tax_post_discount / nullif(rev_post_tax_post_discount, 0) margin_post_tax_post_discount,
      profit_pre_tax_post_discount / nullif(rev_pre_tax_post_discount, 0) margin_pre_tax_post_discount,
      profit_pre_tax_pre_discount / nullif(rev_pre_tax_pre_discount, 0) margin_pre_tax_pre_discount
   from
      monthly_location_rev_and_cogs_and_profit
)
--     select * from monthly_location_rev_and_cogs_and_profit_and_margin
,
monthly_gm_analysis as
(
   select
      year_month,
      shopid,
      companyid,
      -- CHOOSE TO USE PRE TAX POST DISCOUNT
      rev_pre_tax_post_discount,
      lag(rev_pre_tax_post_discount) over (partition by shopid order by year_month) lag_rev_pre_tax_post_discount,
      cogs,
      profit_pre_tax_post_discount,
      margin_pre_tax_post_discount,
      count(distinct year_month) over (partition by shopid ) month_available,
      sum(cogs) over (partition by shopid order by year_month rows between 2 preceding and current row ) sum_cogs_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 1 preceding and current row ) avg_margin_2m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 2 preceding and current row ) avg_margin_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 5 preceding and current row ) avg_margin_6m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month ) avg_margin_all
   from
      monthly_location_rev_and_cogs_and_profit_and_margin
)
select
   *,
   (
      rev_pre_tax_post_discount - lag_rev_pre_tax_post_discount
   )
   / nullif(lag_rev_pre_tax_post_discount, 0) rev_change
from
   monthly_gm_analysis
   where shopid not in ("5e50449d6c0a910843356fa3",
                        "5e666af1aa809b085a98a646",
                        "56cf846ee38179985229e59e",
                        "5e6d0948b987950831b5092c",
                        "5ab806d6c182bb072e0b0e01",
                        "617b7da0e5cb65322b32781a",
                        "56cf84e4e38179985229e59f",
                        "5ee7c6a938fa9a08c8b7939c",
                        "5eebecd661ea0b08cdbcc386",
                        "56cf855ce38179985229e5a0",
                        "5ef12ad7bb5e7a08c5b3f4f3",
                        "5f1108355ae12008f4fef76e",
                        "5f123a5d12497908d2888dde",
                        "5f2309d579373608c93e6742",
                        "5f341ed02fdd5608e0821b45",
                        "5f8870200260fe08fe708db5",
                        "5f8870200260fe08fe708db6",
                        "5fac6a0b0643cc08efd1d327",
                        "56cf855ce38179985229e5a1",
                        "5d238c1955be2e07f5ee66e5",
                        "57b35a5d920f401e40146301",
                        "603d1a9eb9d7dc08cca27203",
                        "603d1a9eb9d7dc08cca27204",
                        "6046544f61e17808cff96fdb",
                        "6046544f61e17808cff96fdc",
                        "57b35aa8920f401e40146334",
                        "57b6b145920f4045cbe5fe81",
                        "6059219abe32aa08fa3a11dc",
                        "57b6b36a920f4046337f2618",
                        "58a7433f819d9e6ebd427617",
                        "60c0ffd6331cab296f02f49a",
                        "610c0a0d381ba0747c981ce9",
                        "611c0c98127cdd740cff94db",
                        "5a43f72b819d9e3ef9373675",
                        "611c0cc403d2e97dc2c159b0",
                        "5e6d0ac5b987950831b513e3",
                        "5a4563f1819d9e4ae6207e06",
                        "611c0ce7649819050ed4906b",
                        "611c0d02f572ec7882d64eea",
                        "611c0d26d5f1e86542fe3b8b",
                        "61672305b5e382719c7e4c2a",
                        "616dee0e4497e822dae13b21",
                        "616e5b13dd10fe4d3f34fd05",
                        "616e5b13dd10fe4d3f34fd06",
                        "619834e921c8017e23135674",
                        "5ab2d1a5819d9e21137cec3c",
                        "61bb746b82fddc50e058d711",
                        "61c3be757511d81170a29d51",
                        "61d49080eec1b51a3e33dc39",
                        "61d8912bdd71d40c2b68c5b3",
                        "61ba4efb9536472be6a8baa9",
                        "61d895fb3efbfb6da5210ae3",
                        "61d8aaf527205210191d7a1e",
                        "61e72d51073c4a79cfeccb62",
                        "61fdab9eae59d05c49422c3a",
                        "6202bd1c15425361f688aabf",
                        "6202bd1c15425361f688aac0",
                        "5b9f8e1db93307080d35b85f",
                        "5c2c250eabe12808989ac2f7",
                        "58be5603c182bb1f9bcd2ad5",
                        "5c8be64665a6a40808118621",
                        "606761051436a608e166db19",
                        "5ce2eb562d4dcf07fb7d0c5f",
                        "6176fa9eda707b1faba5f717",
                        "5b517867485da70830fcc04e",
                        "5b9f841a17b6e00812fe71f3",
                        "5da023586d8ce1082b9c91bd",
                        "5bbce3db420c21083300e410",
                        "5d238c1955be2e07f5ee66e4",
                        "5b209459062bd807b142a1f6",
                        "5cb1a5839fbb5f0818f6de55",
                        "5cda1adb183d6a0801347027",
                        "5be3903b79015e083194aa70",
                        "5d1ba174cdbd6d0803c99cfd",
                        "5a8a6eb8819d9e78641ab6ec",
                        "5db863825a04a2083b464083",
                        "5d8a7fb3a3274308311743d5",
                        "5d8a7fb3a3274308311743d6",
                        "5d9e425d52d15b0824a8a5d5",
                        "5dd6da52ce4015084c72e8db",
                        "5e07f51837cdd7082662f3c6",
                        "5e4d849981e3cd08326b6c14",
                        "5dfd2308fa462a086a8bfab8",
                        "5e0ee2710a7b7008346b06b4",
                        "5e4d849981e3cd08326b6c15",
                        "5dfd2308fa462a086a8bfab9",
                        "5e6d0adc12f2e208728f9369",
                        "5e6d0948b987950831b5092b",
                        "5ed589d96c0bf408dcc92d97",
                        "5eebecd761ea0b08cdbcc387",
                        "5ef12ad7bb5e7a08c5b3f4f2",
                        "5f63afd430bd7208ce1ab304",
                        "5f63afd430bd7208ce1ab305",
                        "5bf5ce54062bd807e436b473",
                        "5fac691a715f3808cd0aa82d",
                        "5d9d7fc27e2e46081e7e69f3",
                        "6034287d52503608c0b3ca48",
                        "5dfd40ea521ea90860c91848",
                        "6034287d52503608c0b3ca49",
                        "60636227ec491108d1c970d5",
                        "5a9a2274c182bb4ec00a72ca",
                        "60aea3bb1940943cecec2dbf",
                        "60a2f1610f75543c8e7dde50",
                        "5eb98f2d649a6508f87a6436",
                        "5e62a68447d90a086b8231c0",
                        "5e62a68447d90a086b8231c1",
                        "615b634cac08aa0a794158c4",
                        "60a2db8d0f75543c8e7d513b",
                        "60a2db8d0f75543c8e7d513c",
                        "60a4051e67406e7edad8ab88",
                        "60c0ff66082dcc61e42ad4a6",
                        "60c0ff66082dcc61e42ad4a7",
                        "6046c57057abe808d1f260fd",
                        "60636227ec491108d1c970d4",
                        "610abcaec3d09859f9b630d1",
                        "606cfaadba253e08c4189a63",
                        "607a321441cfa308f6e65b58",
                        "6197e31b21c8017e2310c10e",
                        "611d457db7c6f80914292394",
                        "610c0a0d381ba0747c981cea",
                        "610abcaec3d09859f9b630d2",
                        "611c0cb3a29e0f598f108a60",
                        "5fff453a81a63608d3a1a5b7",
                        "615237cc0937e52684b6c134",
                        "620b59c97852ca7a7dcc6f05",
                        "6196aa4deb62c87eba4014ad",
                        "619816781f88a1280f8a0c28",
                        "61d7b217a42c2f124d9b76b1",
                        "61cf4b452bc6657512b92d2a",
                        "61ebb060b04dc7510465f774",
                        "61eedaa0b882812f2f46a23d",
                        "61eedade6716dd2a2e0cfce5",
                        "5e6d0ae6e4a25a0834047467",
                        "5dc3315fe0faca07c6dfe849",
                        "5f7e17594445b008d2f63ebd",
                        "5f7f63053dbcad08cc9edbfb",
                        "5fff453a81a63608d3a1a5b8",
                        "603d5425a9ca7208f886d156",
                        "60e48dbbd5596d7c8a1d24dc",
                        "611c0d335901162ff90ef4dd",
                        "613f771aa96c863c00c1517d",
                        "6153b70fd343dc6230c4b67d",
                        "61699cdf032e1c1f920ec70f",
                        "61699cdf032e1c1f920ec710",
                        "61a7b4c6c8c7d85ea1846f14",
                        "61c390cfb863cb3baddb70f4",
                        "61d7b252ae999342ac05499e",
                        "61e6f857c5da34762f1c1fb5",
                        "61ebb00a335a531542342a32",
                        "620b59c97852ca7a7dcc6f04",
                        "6226640755810d3311e8ecd0")
""",engine)

In [4]:
blaze_gm_data['year_month'] = pd.to_datetime(blaze_gm_data['year_month'])

In [5]:
blaze_gm_data

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
0,2022-09-30,62b4b86a23efb97aa2138b78,5faf10dda4594c08c922b600,37584.53,NaN,20218.55,17365.98,0.462051,2,20218.55,0.462051,0.462051,0.462051,0.462051,NaN
1,2022-10-31,62b4b86a23efb97aa2138b78,5faf10dda4594c08c922b600,129066.27,37584.53,66724.87,62341.40,0.483019,2,86943.42,0.472535,0.472535,0.472535,0.472535,2.434026
2,2021-08-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,627440.04,NaN,324162.92,303277.12,0.483356,15,324162.92,0.483356,0.483356,0.483356,0.483356,NaN
3,2021-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,521766.59,627440.04,264822.64,256943.95,0.492450,15,588985.56,0.487903,0.487903,0.487903,0.487903,-0.168420
4,2021-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,613862.25,521766.59,310043.04,303819.21,0.494931,15,899028.60,0.493690,0.490246,0.490246,0.490246,0.176507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-06-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,467480.82,392723.50,227179.35,240301.47,0.514035,11,591080.19,0.516906,0.516706,0.527021,0.527958,0.190356
132,2022-07-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,450610.91,467480.82,214843.93,235766.98,0.523216,11,630617.99,0.518626,0.519010,0.522425,0.527366,-0.036087
133,2022-08-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,448020.93,450610.91,212894.50,235126.43,0.524811,11,654917.78,0.524014,0.520688,0.519463,0.527082,-0.005748
134,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,11,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577


In [6]:
blaze_gm_data[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,6
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,10
2022-03-31,8
2022-04-30,10


In [7]:
blaze_gm_data[blaze_gm_data['year_month'] == '2022-10-31'][['shopid','month_available']].groupby(['shopid']).mean()

,month_available
shopid,
5e7e9d34f795d008e7136fe1,15.0
5faf10dda4594c08c922b618,15.0
5fb5656cd5322c09020a0027,15.0
5fdce3b36c42c608cc9c8b8b,15.0
602c09ee9e113008ed666e63,15.0
607701db4683fe08ef9f2b44,11.0
609d85d3b6a049435d4faf02,11.0
60b913136fc90078bb683ee1,13.0
610daf318c9aff4915efe183,8.0


In [8]:
blaze_gm_data_long = blaze_gm_data[blaze_gm_data['month_available'] >= 5]

In [9]:
blaze_gm_data_long['shopid'].nunique()

10

# 1m 

In [10]:
blaze_gm_data_long

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
2,2021-08-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,627440.04,NaN,324162.92,303277.12,0.483356,15,324162.92,0.483356,0.483356,0.483356,0.483356,NaN
3,2021-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,521766.59,627440.04,264822.64,256943.95,0.492450,15,588985.56,0.487903,0.487903,0.487903,0.487903,-0.168420
4,2021-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,613862.25,521766.59,310043.04,303819.21,0.494931,15,899028.60,0.493690,0.490246,0.490246,0.490246,0.176507
5,2021-11-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,581005.10,613862.25,305157.37,275847.73,0.474777,15,880023.05,0.484854,0.487386,0.486378,0.486378,-0.053525
6,2021-12-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,672536.29,581005.10,345818.44,326717.85,0.485800,15,961018.85,0.480288,0.485169,0.486263,0.486263,0.157539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-06-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,467480.82,392723.50,227179.35,240301.47,0.514035,11,591080.19,0.516906,0.516706,0.527021,0.527958,0.190356
132,2022-07-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,450610.91,467480.82,214843.93,235766.98,0.523216,11,630617.99,0.518626,0.519010,0.522425,0.527366,-0.036087
133,2022-08-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,448020.93,450610.91,212894.50,235126.43,0.524811,11,654917.78,0.524014,0.520688,0.519463,0.527082,-0.005748
134,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,11,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577


In [11]:
# mom margin
gm_distr_mom = blaze_gm_data_long.groupby(['year_month'])['margin_pre_tax_post_discount'].describe().reset_index()
gm_distr_mom.index = pd.to_datetime(gm_distr_mom['year_month'])
gm_distr_mom

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.497434,0.011709,0.487522,0.488357,0.492450,0.504145,0.514694
2021-10-31,2021-10-31,6.0,0.498306,0.022464,0.460658,0.492747,0.499911,0.508064,0.528213
2021-11-30,2021-11-30,6.0,0.496347,0.025522,0.466641,0.479345,0.496106,0.504012,0.538823
2021-12-31,2021-12-31,8.0,0.502336,0.026345,0.473160,0.483350,0.494554,0.522181,0.542658
2022-01-31,2022-01-31,8.0,0.495824,0.052370,0.384271,0.482802,0.498250,0.530127,0.550789
2022-02-28,2022-02-28,9.0,0.527543,0.106037,0.417804,0.483056,0.487122,0.535330,0.791667
2022-03-31,2022-03-31,8.0,0.503760,0.030987,0.450525,0.486485,0.501254,0.524124,0.544857
2022-04-30,2022-04-30,10.0,0.364993,0.334167,-0.537864,0.427829,0.498626,0.512696,0.553917


In [12]:
round(gm_distr_mom[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.51
75%    0.52
dtype: float64

In [13]:
round(blaze_gm_data_long['margin_pre_tax_post_discount'].quantile(0.95),2)

0.56

# 2m

In [14]:
gm_distr_2m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_2m'].describe().reset_index()
gm_distr_2m.index = pd.to_datetime(gm_distr_2m['year_month'])
gm_distr_2m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.498491,0.015407,0.474090,0.494788,0.498411,0.503353,0.521454
2021-11-30,2021-11-30,6.0,0.497326,0.023375,0.463649,0.486774,0.498338,0.504981,0.533518
2021-12-31,2021-12-31,8.0,0.501897,0.025005,0.469901,0.483466,0.498684,0.514974,0.540741
2022-01-31,2022-01-31,8.0,0.499080,0.034802,0.442136,0.480756,0.491239,0.526976,0.543436
2022-02-28,2022-02-28,9.0,0.528119,0.107846,0.401038,0.482763,0.497128,0.539772,0.791667
2022-03-31,2022-03-31,8.0,0.499144,0.035093,0.434164,0.485365,0.493171,0.531387,0.540094
2022-04-30,2022-04-30,10.0,0.433018,0.149375,0.126902,0.443671,0.495790,0.513722,0.549387


In [15]:
round(gm_distr_2m[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.51
75%    0.52
dtype: float64

In [16]:
round(blaze_gm_data_long['avg_margin_2m'].quantile(0.95),2)

0.56

## 3m

In [17]:
gm_distr_3m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_3m'].describe().reset_index()
gm_distr_3m.index = pd.to_datetime(gm_distr_3m['year_month'])
gm_distr_3m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.499983,0.013427,0.486031,0.490830,0.497948,0.504497,0.522830
2021-11-30,2021-11-30,6.0,0.497797,0.018548,0.471607,0.489641,0.497643,0.503665,0.527243
2021-12-31,2021-12-31,8.0,0.502241,0.023884,0.466820,0.486560,0.499938,0.515062,0.536565
2022-01-31,2022-01-31,8.0,0.497820,0.033892,0.442136,0.481977,0.490894,0.521218,0.542186
2022-02-28,2022-02-28,9.0,0.530241,0.103802,0.434025,0.482916,0.493352,0.540734,0.791667
2022-03-31,2022-03-31,8.0,0.498037,0.040410,0.417533,0.486161,0.493715,0.532551,0.541467
2022-04-30,2022-04-30,10.0,0.431072,0.148508,0.126902,0.439959,0.492839,0.518382,0.544701


In [18]:
round(gm_distr_3m[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.50
75%    0.52
dtype: float64

In [19]:
round(blaze_gm_data_long['avg_margin_3m'].quantile(0.95),2)

0.55

## 6m

In [20]:
gm_distr_6m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_6m'].describe().reset_index()
gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
gm_distr_6m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.499983,0.013427,0.486031,0.490830,0.497948,0.504497,0.522830
2021-11-30,2021-11-30,6.0,0.499104,0.016375,0.481184,0.487959,0.497488,0.504514,0.526828
2021-12-31,2021-12-31,8.0,0.503019,0.019653,0.479579,0.488586,0.499938,0.511621,0.533584
2022-01-31,2022-01-31,8.0,0.497538,0.031216,0.442136,0.486461,0.492688,0.514705,0.542186
2022-02-28,2022-02-28,9.0,0.529027,0.103511,0.434025,0.488457,0.491853,0.533989,0.791667
2022-03-31,2022-03-31,8.0,0.497679,0.033461,0.438150,0.485566,0.491203,0.524041,0.539016
2022-04-30,2022-04-30,10.0,0.430122,0.147573,0.126902,0.445262,0.488291,0.516945,0.543300


In [21]:
round(gm_distr_6m[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.50
75%    0.52
dtype: float64

In [22]:
round(blaze_gm_data_long['avg_margin_6m'].quantile(0.95),2)

0.55

## all time

In [23]:
gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])
round(gm_distr_all,3)

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.503,0.019,0.482,0.483,0.510,0.512,0.526
2021-09-30,2021-09-30,5.0,0.500,0.012,0.488,0.493,0.499,0.500,0.520
2021-10-31,2021-10-31,6.0,0.500,0.013,0.486,0.491,0.498,0.504,0.523
2021-11-30,2021-11-30,6.0,0.499,0.016,0.481,0.488,0.497,0.505,0.527
2021-12-31,2021-12-31,8.0,0.503,0.020,0.480,0.489,0.500,0.512,0.534
2022-01-31,2022-01-31,8.0,0.498,0.031,0.442,0.486,0.493,0.515,0.542
2022-02-28,2022-02-28,9.0,0.529,0.103,0.434,0.487,0.492,0.533,0.792
2022-03-31,2022-03-31,8.0,0.497,0.032,0.438,0.488,0.491,0.520,0.536
2022-04-30,2022-04-30,10.0,0.429,0.147,0.127,0.446,0.490,0.511,0.536


In [24]:
round(gm_distr_all[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.50
75%    0.51
dtype: float64

In [25]:
round(blaze_gm_data_long['avg_margin_all'].quantile(0.95),2)

0.54

# Rev change

In [26]:
def gmv_change_variance_point_mapping(x):
    if x < -0.1:
        return -5
    elif x >= -0.1 and x <-0.05:
        return -2.5
    elif x >= -0.05 and x < 0:
        return 0
    elif x >= 0 and x < 0.05:
        return 5
    elif x >= 0.05 and x <= 0.5:
        return 10
    else:
        return numpy.nan
    

## blaze benchmark

In [ ]:
# rev_change_bm = blaze_gm_data_long.groupby(['year_month'])['rev_change'].describe().reset_index()
# rev_change_bm.index = pd.to_datetime(rev_change_bm['year_month'])
# rev_change_bm

In [ ]:
# rev_change_bm[['50%']].plot(kind = 'bar')

In [ ]:
# rev_change_bm[['year_month','50%']]

In [ ]:
# rev_change_bm_with_weight = rev_change_bm[['year_month','50%']]
# rev_change_bm_with_weight['weight'] = [0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
# rev_change_bm_with_weight.columns = ['year_month','rev_change_bm','rev_change_weight']
# rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
# rev_change_bm_with_weight

In [28]:
gm_distr_all['year_month']

year_month
2021-08-31   2021-08-31
2021-09-30   2021-09-30
2021-10-31   2021-10-31
2021-11-30   2021-11-30
2021-12-31   2021-12-31
2022-01-31   2022-01-31
2022-02-28   2022-02-28
2022-03-31   2022-03-31
2022-04-30   2022-04-30
2022-05-31   2022-05-31
2022-06-30   2022-06-30
2022-07-31   2022-07-31
2022-08-31   2022-08-31
2022-09-30   2022-09-30
2022-10-31   2022-10-31
Name: year_month, dtype: datetime64[ns]

## state website published benchmark

In [30]:
# use own bm

rev_change_bm_with_weight = pd.DataFrame([numpy.nan,-0.07, 0.01, -0.07, 0.09,0.07,0.02,0.11,-0.01,-0.05,-0.01,0.07,-0.02,-0.07,-0.04])
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values
rev_change_bm_with_weight['weight'] = [numpy.nan,0.04,0.08,0.12,0.2,0.28,0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
rev_change_bm_with_weight.columns = ['rev_change_bm','year_month','rev_change_weight']
rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
rev_change_bm_with_weight

,rev_change_bm,year_month,rev_change_weight
0,NaN,2021-08-31,NaN
1,-0.07,2021-09-30,0.04
2,0.01,2021-10-31,0.08
3,-0.07,2021-11-30,0.12
4,0.09,2021-12-31,0.20
5,0.07,2022-01-31,0.28
6,0.02,2022-02-28,0.36
7,0.11,2022-03-31,0.44
8,-0.01,2022-04-30,0.52
9,-0.05,2022-05-31,0.60


In [31]:

blaze_gm_data_long_with_rev  = pd.merge(blaze_gm_data_long,rev_change_bm_with_weight,on=['year_month'], how='inner')
blaze_gm_data_long_with_rev['rev_change_var'] = blaze_gm_data_long_with_rev['rev_change'] - blaze_gm_data_long_with_rev['rev_change_bm']
blaze_gm_data_long_with_rev['rev_change_points'] = [gmv_change_variance_point_mapping(n) for n in blaze_gm_data_long_with_rev['rev_change_var']]
blaze_gm_data_long_with_rev['rev_change_total'] = blaze_gm_data_long_with_rev['rev_change_points'] * blaze_gm_data_long_with_rev['rev_change_weight']



In [33]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month']=='2022-10-31']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
115,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5
116,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0
117,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0
118,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0
119,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0
120,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0
121,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0
122,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0
123,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0
124,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0


# inventory

In [34]:
blaze_inv_data_all = pd.read_sql_query("""with base as (
    select *,
    DATE_DIFF(date,purchase_date,DAY) diff,
    current_quantity * unit_cost total_value
    from blaze_data.inventory
    where  lower(product_category) not like '%sample%'
    and lower(product_category) not like '%display%'
    and lower(product_category) not like '%promo%'
    and lower(product_category) not like '%test%'
    -- EXCLUDE SMALL COGS
    and unit_cost > 0.01
    and current_quantity > 0
    and current_quantity < 1000
    and date in ('2022-10-25','2022-10-18','2022-10-11','2022-10-04','2022-10-01')
    and state = 'MA'
),
base_by_group as (
select *,
case when diff < 180 then 'fresh'
     when diff >= 180 and diff < 270 then 'stale'
    else  'bad' end as inventory_group
from base
),
weekly_inv as (
    select date,
           shop_id          shopid,
           company_id       companyid,
           inventory_group,
           sum(total_value) inventory
    from base_by_group
    group by 1, 2, 3,4
)
select
shopid,
companyid,
inventory_group,
avg(inventory) inventory,
last_day('2022-10-31') year_month
from weekly_inv
group by 1,2,3""",engine)

In [35]:
blaze_inv_data_all['year_month'] = pd.to_datetime(blaze_inv_data_all['year_month'])

In [36]:
set(blaze_gm_data['shopid']) - set(blaze_inv_data_all['shopid'])

set()

In [37]:
fresh_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'fresh'][['year_month','shopid','companyid','inventory']]
stale_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'stale'][['year_month','shopid','companyid','inventory']]


fresh_and_stale_inv = fresh_inv.merge(stale_inv, on = ['shopid','companyid','year_month'],how = 'outer')
fresh_and_stale_inv = fresh_and_stale_inv.fillna(0)

fresh_and_stale_inv.columns = ['year_month','shopid','companyid','inventory_fresh','inventory_stale']
fresh_and_stale_inv['inventory_fresh_and_stale'] = fresh_and_stale_inv['inventory_fresh']  + fresh_and_stale_inv['inventory_stale']

In [38]:
fresh_and_stale_inv

,year_month,shopid,companyid,inventory_fresh,inventory_stale,inventory_fresh_and_stale
0,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,236328.913000,39072.750000,275401.663000
1,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,251272.116333,12493.324333,263765.440667
2,2022-10-31,629fd47e3b7a0d3dd6e87a83,629fd47e3b7a0d3dd6e87a68,360.000000,0.000000,360.000000
3,2022-10-31,62aa16506f3b185f56721fb2,62a7b2c8d78cf65bd398cff2,12604.000000,0.000000,12604.000000
4,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,117397.566000,6585.712000,123983.278000
5,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,180317.482333,3912.789000,184230.271333
6,2022-10-31,62a0bd372a174407d97be49f,62a0bd372a174407d97be484,43660.343000,592.000000,44252.343000
7,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,155181.827167,21957.796000,177139.623167
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,269629.819667,17477.120000,287106.939667
9,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,355653.853000,3510.068000,359163.921000


In [39]:
set(blaze_gm_data['shopid']) - set(fresh_and_stale_inv['shopid'])

{'605cef20813d4c08ebf50c06'}

# joined

In [40]:
data_joined = pd.merge(blaze_gm_data_long_with_rev,fresh_and_stale_inv,on=['year_month','shopid','companyid'], how='inner')


In [41]:
data_joined

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000


In [42]:
data_joined['inventory_turnover'] = data_joined['sum_cogs_3m'] / data_joined['inventory_fresh_and_stale'] * 4
data_joined['year_month'] = pd.to_datetime(data_joined['year_month'])

In [43]:
data_joined[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-10-31,10


In [44]:
data_joined[data_joined['year_month'] >= '2021-10-31'].groupby(['year_month'])['inventory_turnover'].describe().reset_index()

,year_month,count,mean,std,min,25%,50%,75%,max
0,2022-10-31,10.0,10.618548,4.089317,6.264098,7.781145,8.69734,13.726951,17.865675


In [45]:
round(data_joined['inventory_turnover'].quantile(0.95),2)

17.02

In [ ]:
# data_joined_week = pd.merge(blaze_gm_data_long_with_rev,blaze_inv_data,on=['shopid','companyid'], how='inner')

# data_joined_week['inventory_turnover'] = data_joined_week['sum_cogs_3m'] / data_joined_week['inventory'] * 4
# data_joined_week['year_month'] = pd.to_datetime(data_joined_week['year_month'])

# data_joined_week

# UW

In [46]:
recent = data_joined[data_joined['year_month'] == '2022-10-31']
recent = recent.reset_index(drop = True)

In [47]:
recent[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-10-31,10


In [48]:
recent

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448


In [ ]:
# recent_week = data_joined_week[data_joined_week['year_month'] == '2022-04-30']
# recent_week = recent_week.reset_index(drop = True)

# recent_week[['shopid','inventory_turnover']].groupby(['shopid']).describe()

## gm % score

In [49]:
def get_gm_perc_scores(threshold,gm_3,gm_6,gm_all):
    # 3m score
    if gm_3 <= threshold[0][0]:
        score_3m = -2
    elif threshold[0][0] < gm_3 <= threshold[0][1]:
        score_3m = 0
    elif threshold[0][1] < gm_3 <= threshold[0][2]:
        score_3m = 5
    elif threshold[0][2] < gm_3 <= threshold[0][3]:
        score_3m = 5
    else:
        score_3m = 0
    # 6m score
    if gm_6 <= threshold[1][0]:
        score_6m = -2
    elif threshold[1][0] < gm_6 <= threshold[1][1]:
        score_6m = 0
    elif threshold[1][1] < gm_6 <= threshold[1][2]:
        score_6m = 5
    elif threshold[1][2] < gm_6 <= threshold[1][3]:
        score_6m = 5
    else:
        score_6m = 0
    # all time score
    if gm_all <= threshold[2][0]:
        score_all = -2
    elif threshold[2][0] < gm_all <= threshold[2][1]:
        score_all = 0
    elif threshold[2][1] < gm_all <= threshold[2][2]:
        score_all = 5
    elif threshold[2][2] < gm_all <= threshold[2][3]:
        score_all = 5
    else:
        score_all = 0
    return score_3m,score_6m,score_all
    

In [50]:
gm_threshold = [[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7]]

In [ ]:
#gm_threshold = [[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7]]

In [ ]:
#gm_threshold = [[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7]]

In [ ]:
#gm_threshold = [[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7]]

In [51]:
# use 3,6,9m
gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['avg_margin_3m'],row['avg_margin_6m'],row['avg_margin_all']),axis = 1)
recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



In [ ]:
# # use 1,2,3m
# gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['margin_pre_tax_post_discount'],row['avg_margin_2m'],row['avg_margin_3m']),axis = 1)
# recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_1m_score', 'gm_2m_score','gm_3m_score'])],axis=1)



In [52]:
recent_with_gm

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675,5,5,5
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707,-2,-2,-2
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5


## inventory turnover score

In [53]:
recent_with_gm['turnover_score'] = [10 if (recent_with_gm['inventory_turnover'][i] >= 6 and recent_with_gm['inventory_turnover'][i] <= 26) else 0 for i in range(len(recent_with_gm))]



In [54]:
recent_with_gm

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675,5,5,5,10
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707,-2,-2,-2,10
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10


## rev change score

In [ ]:
#blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['shopid'] == '60b913136fc90078bb683ee1']

In [55]:
rev_change_score = blaze_gm_data_long_with_rev[['shopid','rev_change_total']].groupby('shopid').sum().reset_index()

In [56]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
rev_change_score.sort_values(by = 'rev_change_total')


,shopid,rev_change_total,rev_change_score
0,5e7e9d34f795d008e7136fe1,-1.4,-1.4
5,607701db4683fe08ef9f2b44,29.0,10.0
4,602c09ee9e113008ed666e63,29.8,10.0
8,610daf318c9aff4915efe183,31.0,10.0
3,5fdce3b36c42c608cc9c8b8b,33.5,10.0
9,623e39dfbcd2bd3c1fb82f46,35.2,10.0
2,5fb5656cd5322c09020a0027,35.8,10.0
1,5faf10dda4594c08c922b618,38.6,10.0
7,60b913136fc90078bb683ee1,40.5,10.0
6,609d85d3b6a049435d4faf02,50.6,10.0


## gm $ score

In [57]:
gm_dollar_score = blaze_gm_data_long_with_rev[['shopid','profit_pre_tax_post_discount']].groupby('shopid').mean().reset_index()

In [58]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['profit_pre_tax_post_discount'][i] >= 200000 else 0 for i in range(len(rev_change_score))]



In [59]:
recent_with_gm_and_rev_change = pd.merge(recent_with_gm,rev_change_score[['shopid','rev_change_score']],on=['shopid'], how='inner')
recent_with_gm_and_rev_change

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675,5,5,5,10,-1.4
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707,-2,-2,-2,10,10.0
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0


In [60]:
final = pd.merge(recent_with_gm_and_rev_change,gm_dollar_score[['shopid','gm_dollar_score']],on=['shopid'], how='inner')



In [61]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit_raw'] = [round(min(final['sum_cogs_3m'][i],final['inventory_fresh'][i]),0) for i in range(len(final))]
final['potential_credit_limit'] = round(final['credit_limit_raw'],-4) 
final['initial_credit_limit'] = final['potential_credit_limit'] / 2                         



In [62]:
final

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.599181,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675,5,5,5,10,-1.4,15,15,15,23.6,155182.0,160000.0,80000.0
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.461421,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707,-2,-2,-2,10,10.0,0,-6,-6,14.0,269630.0,270000.0,135000.0
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0


In [63]:
final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
       'margin_pre_tax_post_discount', 'sum_cogs_3m',
       'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
       'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
       'total_gm_score','turnover_score', 'rev_change_score',  'total_score','credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']]


,year_month,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,437667.52,503458.10,0.534953,1435174.27,0.531575,0.531181,0.519522,0.502901,359163.921000,15.983502,5,5,5,15,15,15,10,10.0,35.0,355654.0,360000.0,180000.0
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,251999.86,265496.52,0.513040,755322.35,0.502620,0.501870,0.504403,0.496247,206025.113000,14.664666,5,5,0,10,15,15,10,10.0,35.0,200066.0,200000.0,100000.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,161323.42,175887.81,0.521595,512114.76,0.516592,0.517596,0.502646,0.491483,246557.779219,8.308231,5,5,0,10,0,10,10,10.0,30.0,232326.0,230000.0,115000.0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,168858.98,211206.13,0.555710,502076.65,0.549431,0.548800,0.537614,0.525731,243417.557000,8.250459,5,5,5,15,0,15,10,10.0,35.0,235230.0,240000.0,120000.0
4,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,239024.96,365484.37,0.604597,791179.75,0.599181,0.594942,0.582123,0.554735,177139.623167,17.865675,5,5,5,15,15,15,10,-1.4,23.6,155182.0,160000.0,80000.0
5,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,77877.82,88243.34,0.531199,223910.36,0.532557,0.536044,0.518130,0.420323,123983.278000,7.223889,5,5,-2,8,0,8,10,10.0,28.0,117398.0,120000.0,60000.0
6,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,105636.53,132821.20,0.557001,288509.10,0.555421,0.554925,0.544958,0.494061,184230.271333,6.264098,5,5,0,10,0,10,10,10.0,30.0,180317.0,180000.0,90000.0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,248635.27,281790.24,0.531253,719671.18,0.515225,0.512478,0.502954,0.495582,263765.440667,10.913806,5,5,0,10,0,10,10,10.0,30.0,251272.0,250000.0,125000.0
8,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,227042.19,171869.03,0.430845,547276.57,0.461421,0.454801,0.431704,0.432142,287106.939667,7.624707,-2,-2,-2,-6,0,-6,10,10.0,14.0,269630.0,270000.0,135000.0
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,194619.58,226440.81,0.537787,625605.73,0.533178,0.530389,0.524699,0.528190,275401.663000,9.086448,5,5,5,15,0,15,10,10.0,35.0,236329.0,240000.0,120000.0


In [64]:
final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
       'margin_pre_tax_post_discount', 'sum_cogs_3m',
       'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
       'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
       'total_gm_score','turnover_score', 'rev_change_score',  'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']].to_csv('blaze_ma_nov_all.csv')


# Proposal Analysis

In [65]:
final['total_score'].describe()

count    10.000000
mean     29.560000
std       6.658528
min      14.000000
25%      28.500000
50%      30.000000
75%      35.000000
max      35.000000
Name: total_score, dtype: float64

In [66]:
pass_list = final[final['total_score'] >= 24]

In [67]:
final_pass = pass_list[(pass_list['profit_pre_tax_post_discount'] >= 25000) & (pass_list['potential_credit_limit'] >= 30000)]
final_pass = final_pass.reset_index(drop = True)

In [68]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0
4,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0
5,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0
6,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0
7,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0


In [69]:
def calculate_interest_rate(score, full_score):
    score_ratio = score / full_score
    placeholder = (1 + 0.5 * (1 - score_ratio)) * 0.015
    rate = placeholder * 12
    return round(placeholder, 4), round(rate, 4)

In [70]:
final_pass['annual_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[1] for i in range(len(final_pass))]
final_pass['monthly_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[0] for i in range(len(final_pass))]



In [73]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit,annual_rate,monthly_rate
0,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531575,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0,0.1800,0.0150
1,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.502620,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0,0.1800,0.0150
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.516592,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0,0.1929,0.0161
3,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.549431,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0,0.1800,0.0150
4,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.532557,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0,0.1980,0.0165
5,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.555421,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0,0.1929,0.0161
6,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.515225,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0,0.1929,0.0161
7,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.533178,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0,0.1800,0.0150


In [75]:
final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
       'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
       'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
       'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
       'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
       'monthly_rate']]

,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit,annual_rate,monthly_rate
0,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,437667.52,503458.10,0.534953,1435174.27,0.531181,0.519522,0.502901,359163.921000,15.983502,5,5,5,15,15,15,10,10.0,35.0,355654.0,360000.0,180000.0,0.1800,0.0150
1,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,251999.86,265496.52,0.513040,755322.35,0.501870,0.504403,0.496247,206025.113000,14.664666,5,5,0,10,15,15,10,10.0,35.0,200066.0,200000.0,100000.0,0.1800,0.0150
2,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,161323.42,175887.81,0.521595,512114.76,0.517596,0.502646,0.491483,246557.779219,8.308231,5,5,0,10,0,10,10,10.0,30.0,232326.0,230000.0,115000.0,0.1929,0.0161
3,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,168858.98,211206.13,0.555710,502076.65,0.548800,0.537614,0.525731,243417.557000,8.250459,5,5,5,15,0,15,10,10.0,35.0,235230.0,240000.0,120000.0,0.1800,0.0150
4,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,77877.82,88243.34,0.531199,223910.36,0.536044,0.518130,0.420323,123983.278000,7.223889,5,5,-2,8,0,8,10,10.0,28.0,117398.0,120000.0,60000.0,0.1980,0.0165
5,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,105636.53,132821.20,0.557001,288509.10,0.554925,0.544958,0.494061,184230.271333,6.264098,5,5,0,10,0,10,10,10.0,30.0,180317.0,180000.0,90000.0,0.1929,0.0161
6,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,248635.27,281790.24,0.531253,719671.18,0.512478,0.502954,0.495582,263765.440667,10.913806,5,5,0,10,0,10,10,10.0,30.0,251272.0,250000.0,125000.0,0.1929,0.0161
7,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,194619.58,226440.81,0.537787,625605.73,0.530389,0.524699,0.528190,275401.663000,9.086448,5,5,5,15,0,15,10,10.0,35.0,236329.0,240000.0,120000.0,0.1800,0.0150


In [76]:
final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
       'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
       'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
       'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
       'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
       'monthly_rate']].to_csv('blaze_ma_nov_approval.csv')

# Summary

In [77]:
print('Total number of shop to UW: ' + str(len(final))+ ' (have at least 6m of data)')
print('Total number of shops approved: ' + str(len(final_pass)))
print('The approval rate is: ' + str(round(len(final_pass)/len(final),2)))
print('Total Final Limit rounded: '+ str(sum(final_pass['potential_credit_limit'])))
print('Average Monthly Rate: ' + str(numpy.mean(final_pass['monthly_rate'])))

Total number of shop to UW: 10 (have at least 6m of data)
Total number of shops approved: 8
The approval rate is: 0.8
Total Final Limit rounded: 1820000.0
Average Monthly Rate: 0.015600000000000001


# ~ END ~